In [1]:
!pip install catboost

     |████████████████████████████████| 65.9MB 66kB/s 


In [2]:
import pandas as pd
import numpy as np
import gc

In [3]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
submission = pd.read_csv('SampleSubmission.csv')

In [4]:
true_values = []
for v in test.values:
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  for k in test.columns[8:][index]:
    true_values.append(v[0] + ' X ' + k)

In [5]:
#Store Products ID in Multidimensional array
products =train[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]
products_t =test[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]

#Axis Transformation
train = train.melt(id_vars=train.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )
test = test.melt(id_vars=test.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )

#A merger
train['combiner']='x'

#Set
data=pd.concat([train,test],sort=False).reset_index(drop=True)

train=data[data.combiner.notnull()].reset_index(drop=True);test=data[data.combiner.isna()].reset_index(drop=True)
train.drop('combiner', inplace=True, axis=1);test.drop(['Label','combiner'], inplace=True, axis=1)

#Concatenate new column towards submission requirement
train['ID X PCODE'] = train['ID'] + ' X ' + train['PCODE'];test['ID X PCODE'] = test['ID'] + ' X ' + test['PCODE']

#Re-arrange column (not necessary)
train=train[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE',  'Label']]
test=test[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE']]


In [6]:
train['date1'] = train['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
train['date2'] = train['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
train['date3'] = train['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
train.drop('join_date', axis=1, inplace=True)

test['date1'] = test['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
test['date2'] = test['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
test['date3'] = test['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
test.drop('join_date', axis=1, inplace=True)

train['date_diff'] = train['date3'] - train['birth_year']
test['date_diff'] = test['date3'] - test['birth_year']

In [7]:
train

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label,date1,date2,date3,date_diff
0,4WKQSBB,F,M,1987,1X1H,2A7I,T4MS,P5DA,4WKQSBB X P5DA,0,1.0,2.0,2019.0,32.0
1,CP5S02H,F,M,1981,UAOD,2A7I,T4MS,P5DA,CP5S02H X P5DA,0,1.0,6.0,2019.0,38.0
2,2YKDILJ,M,U,1991,748L,QZYX,90QI,P5DA,2YKDILJ X P5DA,0,1.0,6.0,2013.0,22.0
3,2S9E81J,M,M,1990,1X1H,BP09,56SI,P5DA,2S9E81J X P5DA,0,1.0,8.0,2019.0,29.0
4,BHDYVFT,M,M,1990,748L,NO3L,T4MS,P5DA,BHDYVFT X P5DA,0,1.0,8.0,2019.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611767,8SBA93U,F,M,1974,748L,2A7I,T4MS,ECY3,8SBA93U X ECY3,0,1.0,9.0,2019.0,45.0
611768,Y73ETTO,M,M,1987,XX25,Q6J6,90QI,ECY3,Y73ETTO X ECY3,0,1.0,11.0,2018.0,31.0
611769,MVXG2LC,M,M,1985,BOAS,6PE7,T4MS,ECY3,MVXG2LC X ECY3,0,1.0,2.0,2019.0,34.0
611770,BUQ8DQN,M,M,1989,UAOD,2A7I,T4MS,ECY3,BUQ8DQN X ECY3,0,1.0,7.0,2019.0,30.0


In [8]:
from catboost import CatBoostClassifier
model = CatBoostClassifier()
model.fit(train.drop(columns=['ID', 'Label', 'ID X PCODE']), train['Label'], cat_features=['sex','marital_status','branch_code','occupation_code','occupation_category_code', 'PCODE'])

Learning rate set to 0.159517
0:	learn: 0.4720387	total: 843ms	remaining: 14m 1s
1:	learn: 0.3400401	total: 1.5s	remaining: 12m 27s
2:	learn: 0.2616355	total: 2.04s	remaining: 11m 17s
3:	learn: 0.2182267	total: 2.65s	remaining: 10m 58s
4:	learn: 0.1840738	total: 3.26s	remaining: 10m 49s
5:	learn: 0.1687360	total: 3.92s	remaining: 10m 49s
6:	learn: 0.1572428	total: 4.46s	remaining: 10m 32s
7:	learn: 0.1477295	total: 5.02s	remaining: 10m 22s
8:	learn: 0.1434123	total: 5.57s	remaining: 10m 12s
9:	learn: 0.1378272	total: 6.06s	remaining: 9m 59s
10:	learn: 0.1338390	total: 6.63s	remaining: 9m 56s
11:	learn: 0.1305040	total: 7.16s	remaining: 9m 49s
12:	learn: 0.1284664	total: 7.87s	remaining: 9m 57s
13:	learn: 0.1270367	total: 8.44s	remaining: 9m 54s
14:	learn: 0.1258030	total: 8.95s	remaining: 9m 47s
15:	learn: 0.1248691	total: 9.54s	remaining: 9m 46s
16:	learn: 0.1237787	total: 10.1s	remaining: 9m 41s
17:	learn: 0.1230524	total: 10.8s	remaining: 9m 48s
18:	learn: 0.1225618	total: 11.4s	rem

In [9]:
proba = model.predict_proba(test.drop(columns=['ID','ID X PCODE'], axis=1))[:, 1]

In [10]:
df_answer = test[['ID X PCODE']]
df_answer['Label'] = proba

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
df_answer.columns = ['ID X PCODE', 'Label']
for i in range(df_answer.shape[0]):
  if df_answer['ID X PCODE'].iloc[i] in true_values:
    df_answer['Label'].iloc[i] = 1.0

df_answer.reset_index(drop=True, inplace=True)
df_answer.to_csv('submission3.csv', index=False)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


OK